In [122]:
import warnings

import os
import copy
import torch
import config as cfg
import torch.nn as nn
from torch.optim import SGD
from models.simpleNet import Net
import sklearn.metrics as metrics
from torch.utils.data import DataLoader

warnings.filterwarnings(action='ignore')
import torch.utils.tensorboard as tb
from datamanager import *

def fgsm(signal, epsilon, gradient, aid):
        if epsilon == 0:
            return signal
        if aid == True:
            aid = -1
        else:
            aid = 1
        perturbation = torch.mul(gradient.sign(), aid*epsilon)
        perturbed_signal = signal + perturbation
        return perturbed_signal
    
class Client():
    def __init__(self, client_id:str, model:nn.Module, data_info:dict=None, device:str=cfg.DEVICE):
        self.id                         = client_id
        #self.cfg = cfg
        
        self.__model                    = None
        self.T                          = 1
        self.device                     = device
        
        self.train_info, self.test_info = data_info['train'], data_info['test'] # 함수화하기
        self.trainset, self.testset     = FEMNIST(self.train_info), FEMNIST(self.test_info)
        self.selected_rounds            = []
        
        self.aid                        = torch.zeros((1, 784))
        self.epsilon                    = 0.1
        
    @property
    def model(self):             
        return self.__model

    @model.setter
    def model(self, model):
        self.__model = model
    
    def __len__(self):
        return len(self.trainset)
    
    def setup(self):
        self.train_loader = DataLoader(self.trainset, batch_size=16, shuffle=True)
        self.test_loader  = DataLoader(self.testset, batch_size=16, shuffle=False)
        self.optimizer    = SGD(self.model.parameters(), lr=0.001)     # TODO: utils.get_optimizer(cfg['optim']:str)
        self.criterion    = nn.CrossEntropyLoss()                     # TODO: utils.get_loss(cfg['loss']:str)
        self.epochs       = 10
    
    def local_train(self)->None:
        self.model.train()
        self.model.to(self.device)
        # TRAINING
        for epoch in range(self.epochs):
            for idx, batch in enumerate(self.train_loader):
                self.optimizer.zero_grad()
                X, Y = batch
                X    = X + self.aid
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)/self.T
                loss = self.criterion(pred, Y)
                loss.backward()
                self.optimizer.step()
                if "cuda" in self.device : torch.cuda.empty_cache()
        # TESTING
        self.get_aid()
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.train_loader):
                X, Y = batch
                X    = X + self.aid
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X) / self.T
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                if "cuda" in self.device : torch.cuda.empty_cache()
            train_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            train_loss = np.average(loss_trace)
            self.model.to('cpu')
        
        print(f'=== Client {self.id} Finished Training {len(self)} samples with T = {self.T} ===')
        print(f'client:{self.id} | Train Acc:{train_acc*100:.2f} | Train Loss:{train_loss:.4f}')
    
    def get_aid(self):
        self.model.to(self.device)
        tmp = torch.zeros(self.aid.shape)
        for idx, batch in enumerate(self.train_loader):
            self.optimizer.zero_grad()
            X, Y = batch
            X    = X + self.aid
            X, Y = X.to(self.device), Y.to(self.device)
            X.requires_grad_(True)
            
            pred = self.model(X) / self.T
            loss = self.criterion(pred, Y)
            loss.backward()
            
            gradient = (-1 * X.grad.data.sign() * self.epsilon).to('cpu')
            tmp = torch.cat([tmp, gradient], dim=0)
            if "cuda" in self.device : torch.cuda.empty_cache()
        self.aid = torch.mean(tmp, dim=0).reshape(1,-1)
        
    
    def local_test(self):
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.test_loader):
                X, Y = batch
                X    = X + self.aid
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X) / self.T
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                if "cuda" in self.device : torch.cuda.empty_cache()
            test_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            test_loss = np.average(loss_trace)
            print(f'client:{self.id} | Test Acc:{test_acc*100:.2f} | Test Loss:{test_loss:.4f}')
            self.model.to('cpu')

In [123]:
import os
import copy
import utils
import torch
import numpy as np
from models import *
from server import *
import config as cfg
import torch.nn as nn
from tqdm import tqdm
from datamanager import *
from datetime import datetime
from models.simpleNet import Net
import sklearn.metrics as metrics
from collections import OrderedDict
from multiprocessing import Process
from torch.utils.data import DataLoader

class Server():
    def __init__(self, DM_dict:dict, args_dict:dict, algorithm:str=None):
        self.train_DM       = DM_dict['train']
        self.test_DM        = DM_dict['test']
        self.clients        = None
        self.args_dict      = args_dict
        self.device         = cfg.DEVICE
        self.global_model   = utils.get_model(args_dict['model'])
        self.criterion      = nn.CrossEntropyLoss()              # TODO: utils.get_loss(cfg['loss']:str)
        self.mp_flag        = False
        self.round          = 0
        self.rounds         = 10
        self.T_type         = args_dict['T_type']
        self.T              = []
        self.TB_WRITER      = tb.SummaryWriter(f'./tensorboard/{str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))}_{args_dict["model"]}_{args_dict["dataset"]}_{cfg.T_dict[self.T_type]}')
        self.global_aid     = torch.zeros((1,784))
    def setup(self):
        self.clients    = self.create_clients()
        self.data       = FEMNIST(self.test_DM.global_test_data)
        self.dataloader = DataLoader(self.data, batch_size=256, shuffle=False)
        self.transmit_model()
        self.setup_clients()
        
    def create_clients(self, n_users:int=35):
        self.user_ids   = self.test_DM.users
        self.user_ids   = np.random.choice(self.user_ids, n_users, replace=False)
        for i in range(n_users):
            self.T.append(cfg.T_dict[self.T_type])
        self.mean_T = np.average(self.T)
        # TODO: T 종류별로 미리 정해두기
        
        clients         = {}
        for idx, user in enumerate(self.user_ids): 
            data_info           = {'train':self.train_DM.data[user],\
                                   'test' : self.test_DM.data[user]}
            
            clients[user]       = Client(client_id=user, model=self.global_model, data_info=data_info, device=self.device)
            clients[user].model = copy.deepcopy(self.global_model)
            clients[user].T     = self.T[idx]
        return clients
    
    
    def setup_clients(self)->None:
        for k, client in tqdm(enumerate(self.clients), leave=False):
            self.clients[client].setup()
    
    
    def transmit_model(self, sampled_clients:list=None)->None:
        if sampled_clients == None:   
            for client in tqdm(self.clients, leave=False):
                self.clients[client].model.load_state_dict(copy.deepcopy(self.global_model.state_dict()))
                self.clients[client].aid = copy.deepcopy(self.global_aid)
        else:
            for client in tqdm(sampled_clients, leave=False):
                # self.clients[client].model.load_state_dict(copy.deepcopy(self.global_model.state_dict()))
                self.clients[client].aid = copy.deepcopy(self.global_aid)


    def sample_clients(self, n_participant:int=10)->np.array:
        assert n_participant <= len(self.user_ids), "Check 'n_participant <= len(self.clients)'"
        selected_clients = np.random.choice(self.user_ids, n_participant, replace=False) 
        for client in selected_clients:
            self.clients[client].selected_rounds.append(self.round)
        return selected_clients

    
    def train_selected_clients(self, sampled_clients:list)->None:
        total_sample = 0
        for client in tqdm(sampled_clients, leave=False):
            self.clients[client].local_train()
            total_sample += len(self.clients[client])
        return total_sample
    
    
    def mp_train_selected_clients(self, procnum:int, client:str)->None:
        self.clients[client].local_train()
    
    
    def test_selected_models(self, sampled_clients):
        for client in sampled_clients:
            self.clients[client].local_test()


    def mp_test_selected_models(self, procnum:int, client:str):
        self.clients[client].local_test()
    
    
    def average_model(self, sampled_clients, coefficients):
        averaged_weights = OrderedDict()
        tmp = torch.zeros(self.global_aid.shape)
        for it, client in tqdm(enumerate(sampled_clients), leave=False):
            local_weights = self.clients[client].model.state_dict()
            for key in self.global_model.state_dict().keys():
                if it == 0:
                    averaged_weights[key] = coefficients[it] * local_weights[key]
                else:
                    averaged_weights[key] += coefficients[it] * local_weights[key]
            tmp += coefficients[it] * self.clients[client].aid
        self.global_model.load_state_dict(averaged_weights)
        self.global_aid = tmp
    
    
    def update_model(self, train_result:dict, layers:list=None):
        self.received_models, num_samples = [], []
        for result in train_result:
            self.received_models.append(result['model'])
            num_samples.append(result['num_sample'])
        state = self.Algorithm(self.received_models, num_samples, layers)
        self.global_model.load_state_dict(state)


    def train_federated_model(self):
        self.round += 1
        sampled_clients = self.sample_clients()
        print(f"CLIENTS {sampled_clients} ARE SELECTED!\n")
        
        if self.mp_flag:
            print("TRAIN WITH MP!\n")
            procs = []
            selected_total_size = []
            for idx, c in enumerate(sampled_clients):
                selected_total_size.append(len(self.clients[c]))
                proc = Process(target=self.mp_train_selected_clients, args=(idx, c))    # with Multi-Process
                proc.start()
                procs.append(proc)
            for proc in procs:
                proc.join()
            selected_total_size = sum(selected_total_size)
            # with pool.ThreadPool(processes=cpu_count() - 1) as workhorse: # with Threading
            #     selected_total_size = workhorse.map(self.mp_train_selected_clients, sampled_clients)
            
        else:
            print("TRAIN WITH SP!\n")
            selected_total_size = self.train_selected_clients(sampled_clients)

        print("TEST WITH SP!\n")
        self.test_selected_models(sampled_clients)
                
        mixing_coefficients = [len(self.clients[client]) / selected_total_size for client in sampled_clients]
        
        self.average_model(sampled_clients, mixing_coefficients)
        self.transmit_model()
        
        
    def global_test(self):
        self.global_model.eval()
        self.global_model.to(self.device)
        
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.dataloader):
                X, Y = batch
                X    = X + self.global_aid
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.global_model(X) / self.mean_T
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
            self.acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            self.test_loss = np.average(loss_trace)
            self.TB_WRITER.add_scalar(f'Global Test Accuracy', self.acc      , self.round)
            self.TB_WRITER.add_scalar(f'Global Test Loss'    , self.test_loss, self.round)
            print(f'Global Test Result | Acc:{self.acc*100:.2f}, Loss:{self.test_loss:.4f}')
            self.global_model.to('cpu')
    
    
    def test_all_client(self):
        with torch.no_grad():
            for c in self.clients:
                self.clients[c].local_test()
    
    
    def fit(self):
        pass
    
    
    def save_model(self, round=100, tag='', ckpt:list=None):
        model_save_path = f'./checkpoints/{self.args_dict["model"]}_{self.args_dict["dataset"]}_{cfg.T_dict[self.T_type]}'
        
        if not os.path.isdir(model_save_path):
            os.makedirs(model_save_path, exist_ok=True)
        torch.save(self.global_model.state_dict(), f'{model_save_path}/{tag}_{round}.pth')   

In [118]:
server.global_model.eval()
server.global_model.to(server.device)

with torch.no_grad():
    loss_trace, result_pred, result_anno = [], [], []
    for idx, batch in enumerate(server.dataloader):
        X, Y = batch

        X    = X + server.global_aid
        X, Y = X.to(server.device), Y.to(server.device)
        pred = server.global_model(X) / server.mean_T
        loss = server.criterion(pred, Y)
        loss_trace.append(loss.to('cpu').detach().numpy())
        pred_np  = pred.to('cpu').detach().numpy()
        pred_np  = np.argmax(pred_np, axis=1).squeeze()
        Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
        result_pred = np.hstack((result_pred, pred_np))
        result_anno = np.hstack((result_anno, Y_np))
    server.acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
    server.test_loss = np.average(loss_trace)
    # server.TB_WRITER.add_scalar(f'Global Test Accuracy', server.acc      , server.round)
    # server.TB_WRITER.add_scalar(f'Global Test Loss'    , server.test_loss, server.round)
    print(f'Global Test Result | Acc:{server.acc*100:.2f}, Loss:{server.test_loss:.4f}')
    server.global_model.to('cpu')




Global Test Result | Acc:70.63, Loss:1.0268


In [24]:
args = utils.parse_args()
args_dict = utils.args2dict(args)
    
PATH = cfg.DATAPATH['femnist']
    
file_dict = get_files(PATH)
    
TRAIN_DM = DataManager(file_dict['train'], is_train=True, is_flat=args.model=='dnn')
TEST_DM = DataManager(file_dict['test'], is_train=False, is_flat=args.model=='dnn')
DM_dict = {'train':TRAIN_DM,
        'test':TEST_DM}
print("DATA READY")

DATA READY


In [126]:
args_dict['T_type'] = 2

print(f"WORKING WITH {cfg.DEVICE}")
    
server = Server(DM_dict, args_dict)
print("SERVER READY")
    
server.setup()
print('===== ROUND 0 =====\nServer Setup Complete!')

WORKING WITH cuda
SERVER READY


===== ROUND 0 =====
Server Setup Complete!


In [127]:
server.save_model(round=0)
for i in range(50):
    print(f'===== ROUND {i+1} START! =====\n')
    server.train_federated_model()
    server.global_test()
    if (i+1) % 10 == 0:
        server.save_model(round=i+1)

  0%|          | 0/10 [00:00<?, ?it/s]

===== ROUND 1 START! =====

CLIENTS ['4' '28' '34' '25' '8' '29' '24' '27' '20' '22'] ARE SELECTED!

TRAIN WITH SP!



 10%|█         | 1/10 [00:07<01:05,  7.25s/it]

=== Client 4 Finished Training 4955 samples with T = 0.05 ===
client:4 | Train Acc:36.35 | Train Loss:2.5300


 20%|██        | 2/10 [00:19<01:22, 10.27s/it]

...

 60%|██████    | 6/10 [00:40<00:25,  6.33s/it]

=== Client 31 Finished Training 2677 samples with T = 0.05 ===
client:31 | Train Acc:79.45 | Train Loss:0.7888


 70%|███████   | 7/10 [00:47<00:19,  6.42s/it]

=== Client 23 Finished Training 3216 samples with T = 0.05 ===
client:23 | Train Acc:78.95 | Train Loss:0.7476


KeyboardInterrupt: 